In [38]:
import pandas as pd
import os

In [39]:
# # base_dir = '/Users/shannon/science/HMM/HMM_MRI/omni_BIDS/'
data_base_dir = '/Volumes/Seagate/'
# bids_raw_dir = base_dir + 'bids_raw/'
# mriqc_dir = bids_raw_dir + 'mriqc_reports/'
fmriprep_dir = data_base_dir + 'derivatives/fmriprep/'

# change base to location of NSF repo
grant_dir = '/Users/davidhalpern/Documents/GitHub/identifying_causal_subsequent_memory_effects/'

In [46]:
subj_runs_df = pd.read_csv('data/mri_subs_events_wide.csv')

subjects = subj_runs_df.subject.unique()

In [51]:
# load the vox_coord df
df_dir = grant_dir + '/mri_code/roi_info/'
vox_coord_df = pd.read_csv(df_dir +  'vox_coord_mapper.csv')

vox_coord_df.head()

,x,y,z,roi_num_schaeffer,roi_vox_num_schaeffer,roi_num_HarvardOxford_lat,hemisphere_HarvOxford,roi_num_HarvardOxford_bilat
0,0,0,0,0.0,0,0,none,0
1,0,0,1,0.0,1,0,none,0
2,0,0,2,0.0,2,0,none,0
3,0,0,3,0.0,3,0,none,0
4,0,0,4,0.0,4,0,none,0


In [42]:
all_peeps_use_voxels_schaeffer = pd.read_csv(grant_dir + 'mri_code/roi_info/subj_voxels/all_peeps_use_voxels_schaeffer.csv')

In [47]:
all_peeps_use_voxels_schaeffer_long = pd.wide_to_long(
    all_peeps_use_voxels_schaeffer, stubnames='use_voxel_sub', sep='-', 
    i=['x', 'y', 'z', 'roi_num_schaeffer', 'roi_vox_num_schaeffer'], j='sub').reset_index()
all_vox_df = all_peeps_use_voxels_schaeffer_long.drop(
    columns='sub').groupby(
    ['roi_num_schaeffer', 'roi_vox_num_schaeffer', 'x', 'y', 'z']).sum().reset_index()
all_vox_df['use_vox'] = all_vox_df['use_voxel_sub'] == len(subjects)

In [48]:
vox_coord_df_merge = vox_coord_df.merge(all_vox_df, on=['x', 'y', 'z', 'roi_num_schaeffer', 'roi_vox_num_schaeffer'])

In [50]:
for sub_num in subjects:
    peep = 'sub-' + str(sub_num)
    print(peep)
    csv_folder = fmriprep_dir + peep + '/masks/pulls/'
    os.makedirs(csv_folder + 'single_trial_glms/', exist_ok=True)
    for run_num in range(1, 6):
        for trial_num in range(1, 46):
            csv_suffix = 'single_trial_glm_tstat1_run0' + str(run_num) + '_trial' + ('%03d' % trial_num) + '.csv'
            trial_dfs = []
            for roi_num in range(1, 101):
                schaeffer_csv_folder = csv_folder + 'schaeffer/single_trial_glms/roi_' + ('%03d' % roi_num) + '/'
                csv_name = peep + '_schaeffer_roi_' + ('%03d' % roi_num) + '_' + csv_suffix
                trial_df = pd.read_csv(schaeffer_csv_folder + csv_name, names=['voxnum', 'vox_val'], header=0)
                trial_df['roi_num'] = roi_num
                trial_df['study_block'] = run_num
                trial_df['trial_num'] = trial_num
                trial_df['subject'] = sub_num
                trial_dfs.append(trial_df)
            trial_df = pd.concat(trial_dfs)
            trial_xyz_df = trial_df.merge(vox_coord_df_merge, left_on=['roi_num', 'voxnum'], 
                                          right_on=['roi_num_schaeffer', 'roi_vox_num_schaeffer'])
            trial_xyz_df.to_csv(csv_folder + 'single_trial_glms/' + csv_suffix)

sub-11
sub-12
sub-15
sub-16
sub-17
sub-19
sub-20
sub-24
sub-25
sub-26
sub-27
sub-28
sub-29
sub-30
sub-31
sub-32
sub-33
sub-34
sub-35
sub-36
sub-37
sub-38
sub-39
sub-40
sub-42
sub-43
sub-44
sub-45
sub-46
sub-47
sub-48
sub-49
sub-50
sub-51
sub-52
sub-53
sub-54
sub-55
sub-56
sub-57
sub-58
sub-59
sub-60
sub-61
